In [6]:
 #setup the matplotlib graphics library and configure it to show
# figures inline in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import *
# make qutip available in the rest of the notebook
from qutip import *
from matplotlib import animation as animation, pyplot as plt, cm
from IPython import display 
from IPython.display import HTML
from scipy.integrate import simpson, cumtrapz



def Opera(dim,S):
    if S==1:
        jj=jmat(1)
        jx,jy,jz=jmat(1)
        jp=jmat(1,'+')
        jm=jmat(1,'-')
        sx = tensor(qeye(dim), jx)
        sy = tensor(qeye(dim), jy)
        sz = tensor(qeye(dim), jz)
        sm = tensor(qeye(dim), jm)
        sp = sm.dag()
        a = tensor(destroy(dim), qeye(3))
        ad = a.dag()
        ids = tensor(qeye(dim), qeye(3))
    else:
        
        sx = tensor(qeye(dim), sigmax())
        sy = tensor(qeye(dim), sigmay())
        sz = tensor(qeye(dim), sigmaz())
        sm = tensor(qeye(dim), sigmam())
        sp = sm.dag()
        a = tensor(destroy(dim), qeye(2))
        ad = a.dag()
        ids = tensor(qeye(dim), qeye(2))
        
    return sm, sp, sx, sy, sz, a, ad, ids




def compute_expectations(states, observables):
    """
    Compute the expectation values of given observables for a list of states.
    
    Parameters:
    - states: A list of Qobj states, typically the result of a mesolve.
    - observables: A list of Qobj observables for which to compute expectation values.
    
    Returns:
    - A list of lists, where each sublist contains the expectation values of an observable over time.
    """
    # Initialize a list to store the expectation values for each observable
    expectation_values = [[] for _ in observables]

    # Loop over each state
    for state in states:
        # Loop over each observable
        for i, obs in enumerate(observables):
            # Calculate and store the expectation value
            expectation_values[i].append(qutip.expect(obs, state))

    return expectation_values


def generate_c_ops(gammam, gammas, gammaa, gamma_l, n_th_a, sp_th_a, a, sz, sm, sp, include_lasing=False):
    """
    Generate collapse operators based on provided rates and conditions.
    
    Parameters:
    - gammam: Rate for cavity relaxation and excitation.
    - gammas: Rate for qubit relaxation.
    - gammaa: Rate for additional qubit dynamics.
    - gamma_l: Rate for lasing.
    - n_th_a: Thermal occupation number for cavity.
    - sp_th_a: Thermal occupation number for qubit.
    - a: Annihilation operator for the cavity.
    - sz, sm, sp: Pauli operators or equivalent for the qubit.
    - include_lasing: Boolean to include lasing in the collapse operators.
    
    Returns:
    - A list of collapse operators.
    """
    c_ops = []
    # Cavity relaxation
    rate = gammam * (1 + n_th_a)
    if rate > 0.0:
        c_ops.append(sqrt(rate) * a)
    # Cavity excitation, if temperature > 0
    rate = gammam * n_th_a
    if rate > 0.0:
        c_ops.append(sqrt(rate) * a.dag())
    # Qubit relaxation
    rate = gammas
    if rate > 0.0:
        c_ops.append(sqrt(rate) * sz)
    rate = gammaa * (1 + sp_th_a)
    if rate > 0.0:
        c_ops.append(sqrt(rate) * sm)
    rate = gammaa * sp_th_a
    if rate > 0.0:
        c_ops.append(sqrt(rate) * sp)
    # Lasing
    if include_lasing:
        rate = gamma_l
        if rate > 0.0:
            c_ops.append(sqrt(rate) * sm)
    
    return c_ops



    
    
def evolst(H, psi0, t, cops, observables):
    opts = Options(store_states=True)
    result = mesolve(H, psi0, t, cops, [], args={}, options=opts)
    exp_vals = compute_expectations(result.states, observables)
    
    return result, exp_vals





def Cyclevol(rho0, hamiltonian_time_pairs, c_ops_list, observables):
    """
    Perform a sequence of time evolutions on an initial state with different Hamiltonians
    and return individual heat contributions for each term in the collapse operators.

    Parameters:
    - rho0: Initial state (Qobj)
    - hamiltonian_time_pairs: List of tuples (H, tlist) for each evolution segment
    - c_ops_list: List of lists of collapse operators corresponding to each Hamiltonian
    - observables: List of observables (Qobj) to compute expectation values

    Returns:
    - all_states: Concatenated list of states from all evolution segments
    - all_times: Concatenated array of time points from all evolution segments
    - all_exp_vals: Concatenated expectation values for each observable over time
    - EntropyM: List of von Neumann entropy values for mechanical part over time
    - EntropyS: List of von Neumann entropy values for spin part over time
    - total_Q_terms_list: List of lists of individual heat contributions for each term in the collapse operators
    """
    all_states = []
    all_times = array([])
    all_exp_vals = [[] for _ in observables]
    current_state = rho0
    EntropyM = []
    EntropyS = []
    total_Q_terms_list = None  # Changed to None for clearer initialization logic

    for i, ((H, tlist), c_ops) in enumerate(zip(hamiltonian_time_pairs, c_ops_list)):
#         result, exp_vals, Q_terms = evolst(H, current_state, tlist, c_ops, observables)
        result, exp_vals = evolst(H, current_state, tlist, c_ops, observables)

        all_states.extend(result.states)

        if i == 0:
            all_times = array(tlist)
        else:
            # Ensure continuity in time by adding the last time from all_times
            all_times = concatenate((all_times, all_times[-1] + tlist))

        for j, exp_val in enumerate(exp_vals):
            all_exp_vals[j].extend(exp_val)

        for state in result.states:
            EntropyM.append(entropy_vn(state.ptrace(0), base=e))
            EntropyS.append(entropy_vn(state.ptrace(1), base=e))

        current_state = result.states[-1]

    return all_states, all_times, all_exp_vals, EntropyM, EntropyS



def CycleCool(rho0, hamiltonian_time_pairs, c_ops_list, observables, Ncycles):
    
    current_rho = rho0
    t=array([0])
    allst=[current_rho]
    rho_mech = current_rho.ptrace(0)
    rho_spin = current_rho.ptrace(1)
    EntropyM=[entropy_vn(rho_mech, base=e)]
    EntropyS=[entropy_vn(rho_spin, base=e)]
    
   
    
    for i in range(Ncycles):
        
#         if i==0:
#              t=array([0])
        rhot = current_rho.ptrace(0)
        rhoc, tt, exp_vals, entropyM, entropyS = Cyclevol(current_rho, hamiltonian_time_pairs, c_ops_list, observables)    
        EntropyM=concatenate((EntropyM,entropyM), axis=0)
        EntropyS=concatenate((EntropyS,entropyS), axis=0)
        allst=allst+rhoc
        t=concatenate((t, tt + (t[-1] if i > 0 else 0)))

        current_rho=rhoc[-1]
        print(i)
    
    return t, allst, EntropyM, EntropyS




In [92]:
N =50
S=0.5



kb= 20836619120 
wm = 50 * 2 * pi 
fy =  1*10**(6)
g=0.25*wm
wt = 10000
T=1*pi/(wm)
fx = 0.5*wm;
Delta=3*wm

######### Operators ##################
sm, sp, sx, sy, sz, a, ad, ids=Opera(N,S)
z = a+ad
p = 1j*(ad-a)
z2 = z**2
p2 = p**2


######### Damping rates for each mode
Qm=100000
gammam = 1*wm/Qm
# gammam = 1*10**(-1)
gammas = 1/(2.5*10**(-3))
gammaa = 1/(5*10**(-3))
gamma_l = 1/(10**(-6))
n_th_a =5.     # avg number of thermal bath excitation
sp_th_a= 0.0001
n_th = n_th_a

c_ops1 = generate_c_ops(gammam, gammas, gammaa, gamma_l, n_th_a, sp_th_a, a, sz, sp, sm, include_lasing=False)
c_ops2 = generate_c_ops(gammam, gammas, gammaa, gamma_l, n_th_a, sp_th_a, a, sz, sp, sm, include_lasing=True)


######### Time evolutions ########


Tpy = pi/(2*abs(fy))
tfree = 1*pi/wm
tfree2 = 2*tfree 
tlase=5*10**(-5)

# T = tfree + Tpy + tlase
twm=linspace(0,T,500)
tpy = linspace(0, Tpy, 50)
tf1 = linspace(0, tfree, 150)
tf2 = linspace(0, tfree2, 150)
tgl = linspace(0, tlase, 50)


# rho1=steadystate(H, c_ops1)
rho1=tensor(thermal_dm(N,n_th),ket2dm(basis(2,0)))


In [93]:
from joblib import Parallel, delayed
from scipy.io import savemat, loadmat
import numpy as np
import qutip
from tqdm import tqdm
import traceback



def compute_for_Delta_g(i, j, Delta, g):
    try:
        print(f"Processing Delta index {i}, g index {j}")

        H = Delta * (ids - sz) / 2 + wm * ad * a - g * (ids - sz) * (a + ad) / 2
        Hseq = [(H + fx * sx, tf1), (H, tgl)]
        c_ops_seq = [c_ops1, c_ops2]

        observables = [ad * a, z, p, z2, p2, a, a**2, sz, sx, sy, H]

        t, allst, EntropyM, EntropyS = CycleCool(rho1, Hseq, c_ops_seq, observables, 10)
        rho3 = allst[-1]
        exp3 = compute_expectations([rho3], observables)

        return i,j,Delta, g, exp3
    except Exception as e:
        print(f"Error in computation for Delta: {Delta}, g: {g}")
        traceback.print_exc()
        return i,j,Delta, g, None

# Example usage
wm = 50 * 2 * np.pi  # Define your actual value for wm

# Define your range of De|lta and g values
Deltas = np.linspace(0.05 * wm, 16 * wm, 60)
gs = np.linspace(0.05 * wm, 4 * wm, 55)


# Parallelize the computation
results_list = Parallel(n_jobs=-1)(
    delayed(compute_for_Delta_g)(i, j, Delta, g) for i, Delta in tqdm(enumerate(Deltas), total=len(Deltas)) for j, g in enumerate(gs)
)

results = {
    "Delta": np.zeros((len(Deltas), len(gs))),
    "g": np.zeros((len(Deltas), len(gs))),
    "exp3": np.empty((len(Deltas), len(gs)), dtype=object)
}
# Populate the results dictionary with the parallel computation results
for res in results_list:
    if res[4] is not None:
        i, j = res[0], res[1]
        results["Delta"][i, j] = res[2]
        results["g"][i, j] = res[3]
        results["exp3"][i, j] = res[4]

# Save the dictionary to a .mat file
savemat('optimalcoolingresults.mat', results)

100%|█████████████████████████████████████████████| 9/9 [00:08<00:00,  1.07it/s]


'Delta'